# **Deep Learning Project**

**Eliana Battisti - Davide Dalla Stella - Francesco Trono**

*University of Trento*

A.Y. 2020/2021 - Deep Learning Course

Click to open & run in Colab:


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ftrono/DL_Project/blob/main/source_code/MAIN_CODE_UNITO.ipynb)

In [ ]:
#!pip install --pre torch torchvision -f https://download.pytorch.org/whl/nightly/cu102/torch_nightly.html -U

In [19]:
import os
import shutil
import csv
import torch
import torchvision
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
import collections
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
from google.colab import drive
from sklearn.preprocessing import MultiLabelBinarizer

In [20]:
#Set FLAG to 1 only if you want to create the Dataset from 0
FLAG = 1

if os.path.isdir("/content/Dataset") and FLAG == 1:
  shutil.rmtree("/content/Dataset")

In [21]:
#Global variables:
num_classifiers = 12
device = "cuda:0"
#device = "cpu"
localpath = "/content/Dataset/"

**1) Dataset preparation**

In [22]:
#Mounting Google Drive:
drive.mount('/content/drive/', force_remount = False)

#Main path to dataset:
dspath = "/content/drive/My Drive/DL_Project_Team/DL_Project/Dataset"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [23]:
#copy train files in local
from google.colab import files
!mkdir "Dataset"
!unzip "/content/drive/MyDrive/DL_Project_Team/DL_Project/Dataset/dataset.zip" -d "/content/Dataset/" >/dev/null
print("Unzipped.")



Unzipped.


In [24]:
#Parte training split (train / validation e validation_query, test):
#Creare 2 folder diverse, una per Train e l'altra per Validation, entrambe partendo dalla folder Train esistente.

#Scaricare dataset su macchina locale (mantieni struttura sottofolder) (esempio -> https://colab.research.google.com/drive/1oOKXtUUaACKG_P1SZO-PpfygP5_12dX7#scrollTo=h8gyumMki8A6 )
#Parte training split (train / validation):
#Creare 2 folder diverse, una per Train e l'altra per Validation, entrambe partendo dalla folder Train esistente.

#Seguire indicazioni nel TO-DOs. Lavorare tramite bash in locale:
#Scaricare dataset su macchina locale (mantieni struttura sottofolder) (esempio -> https://colab.research.google.com/drive/1oOKXtUUaACKG_P1SZO-PpfygP5_12dX7#scrollTo=h8gyumMki8A6 )

def dataset_preparation(img_root, csv_dir):

    # PART 1

    # directory che porta all'interno della cartella validation
    validation_dir = img_root+"validation"
    train_dir = img_root+"train"  # directory che porta all'interno della cartella train
    n_id = -1
    # contatore per sapere quanti id di persone ho. Parte da -1 così tolgo la riga che contiene solo la legenda

    # per evitare di rischiare di ripetere lo spostamento di immagini ad ogni esecuzione del programma lo
    # spostamento di immagini lo eseguo solo nel caso in cui la cartella non esista così da essere sicuro di
    # spostare le immagini solo alla prima esecuzione
    # controllo se esiste la cartella validation. Se no allora la creo
    if not os.path.isdir(validation_dir):
        os.mkdir(validation_dir)
        # il csv_file mi serve per sapere quanti id di persone ho
        with open(csv_dir) as annotations_train:  # conto il numero di id con un ciclo for
            print(annotations_train)
            # sommo il numero di righe all'interno del csv file
            n_id += sum(1 for row in annotations_train)

        to_move = n_id // 4

        print("Moving 25% of training files in validation")
        # loop che sposta le foto di un quarto delle persone contenute nella cartella
        for i in range(0, to_move):
            # metodo per ottenere la lista coi nomi di tutti i file all'interno di una dir
            # ad ogni ciclo aggiorno la lista così non rischio di spostare file già spostati
            train_list = os.listdir(train_dir)
            # prendo il nome della prima immagine nella lista e mi prendo solo le prime 4 lettere che indicano l'id della persona
            id_to_move = train_list[0][0:4]
            #print("id to move: ", id_to_move)
            for files in train_list:  # scorro tutta la lista dei file
                # controllo se i file iniziano con l'id selezionato
                if files.startswith(id_to_move):
                    # se l'id corrisponde lo sposto
                    shutil.move(train_dir+"/"+files, validation_dir+"/"+files)
                    #print("Moved: ",train_dir+"/"+files, "in: ",validation_dir+"/"+files)

            # just a check to see if I did something  wrong and there are duplicates in the two folders
            '''train_list = os.listdir(train_dir) 
      val_list = os.listdir(validation_dir) 
      if files in train_list and files in val_list:
        file_train = train_list[train_list.index(files)]
        file_val = val_list[val_list.index(files)]
        print("Train file: "file_train,"\nVal file: ",file_val,"\n\n")'''
            # end check

            # just a check to see if I missed to move an image that starts with a certain index
            '''train_list = os.listdir(train_dir) 
      val_list = os.listdir(validation_dir) 
      for train_file in train_dir:
        id_train = train_file[0][0:4]
        for val_file in val_list:
          if val_file.startswith(id_train):
            print("Train file: ",train_file,"\nVal file: ",val_file,"\n\n")'''
            # end check

        print("Training files moved to validation")
    else:
        print("Validation already created.")

    # PART 2

    '''validation_queries_dir = img_root+ "/validation_queries"
  test_dir = img_root + "/test"

  if not os.path.isdir(validation_queries_dir): #Se una delle cartelle non esiste allora assumo che non ne esista nessuna
    if not os.path.isdir(test_dir):
      os.mkdir(validation_queries_dir) #creazione cartelle validation_query e test
      os.mkdir(test_dir)
      validation_list = os.listdir(validation_dir) #lista di file all'interno della cartella validation
      copied_id = [] #lista degli id che ho già copiato in validation list. All'inizio è vuota perchè non ho copiato nessuna immagine
      for file in validation_list:  #scorro la lista di file di validation
        id = file[0:4] #prelevo l'id del file corrente
        if id in copied_id: #se l'id è nell'array allora l'ho già copiato in validation_query quindi quest'altra immagine va copiata in test
          shutil.copy(validation_dir+"/"+file,test_dir+"/"+file) #copia file
        else:
        #se l'id non è presente nell'array allora devo copiare l'immagine in validation_query e inserire l'id nell'array dei già copiati
          shutil.copy(validation_dir+"/"+file,validation_queries_dir+"/"+file)
          copied_id.append(id)'''



In [25]:
class CustomDataset(Dataset):

    # STUDIARSI L'ESEMPIO GIA' FATTO QUI: -> https://pytorch.org/tutorials/beginner/basics/data_tutorial.html , sezione "Creating a Custom Dataset for your files"

    # override:
    # csvfile e imgfolder sono stringhe con la directory corrispettiva
    def __init__(self, imgfolder, train, csvfile=None):

        # creazione dizionario
        self.train = train
        self.imgfolder = imgfolder
        self.dictionary = {}
        self.img_list = os.listdir(imgfolder)
        # numero di file all'interno della cartella contenente le immagini
        self.size = len(self.img_list)
        if csvfile != None:
            with open(csvfile, mode='r') as annotations_train:
                reader = csv.reader(annotations_train, delimiter=',')
                # Salto prima riga per non avere i nomi degli attributi nel dizionario
                next(reader, None)
                for row in reader:
                    # Questo modo ritorna una sublist nel range di index indicato senza contare però l'ultimo elemento!
                    up = row[11:19]
                    down = row[19:]
                    # tengo per la lista solo i primi 10 attributi e quelli relativi ai colori up e down li compatto in due attributi distinti che concateno poi
                    id = int(row[0])

                    row = row[1:11]

                    if '2' in up:  # per le label decido che il loro valore è il corrispettivo dell'index+1. Se nessun valore è ugale a '2' allora ho un multicolor che indico con un valore = all'ultima label+1
                        # concateno attributo upcolor
                        row.append(up.index('2')+1)
                    else:
                        row.append(9)
                    if '2' in down:
                        # concateno attributo downcolor
                        row.append(down.index('2')+1)
                    else:
                        row.append(10)

                    for label in row:
                        # converto in intero le label degli attributi
                        row[row.index(label)] = int(label) - 1
                    self.dictionary[id] = row

    # override:
    # return the element at index idx:
    def __getitem__(self, idx):
        transform = list()
        # transform.append(T.ConvertImageDtype(torch.float))
        transform.append(T.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]),
        )
        transform.append(T.Resize((256, 128)))

        img = Image.open(self.imgfolder+"/"+self.img_list[idx])
        img = T.ToTensor()(img)

        transform = T.Compose(transform)
        img = transform(img)
        if self.train == True:
            transform = list()

            transform.append(T.RandomRotation(5))
            transform.append(T.RandomCrop((256, 128), 10))
            transform.append(T.RandomHorizontalFlip())
            transform = T.Compose(transform)
            img = transform(img)
            # print(self.dictionary[int(self.img_list[idx][0:4])])
            return img, (torch.as_tensor(self.dictionary[int(self.img_list[idx][0:4])]))
        else:
            return img
    # override:

    def __len__(self):
        # return the number of elements that compose the dataset:
        return self.size

In [26]:
def translate_attributes(attributes):
  translated_attributes = {}
  attributes_names = ["age","backpack","bag","handbag","clothes","down","up","hair","hat","gender","upcolor","downcolor"] #traduttore

  translator = [
                ["young", "teenager", "adult", "old"], #index: 0 attribute: age
                ["no", "yes"], #index: 1 attribute: backpack
                ["no", "yes"], #index: 2 attribute: bag
                ["no", "yes"], #index: 3 attribute: handbag
                ["dress", "pants"], #index: 4 attribute: clothes
                ["long lower body clothing", "short"], #index: 5 attribute: down
                ["long sleeve short", "sleeve"], #index: 6 attribute: up
                ["short hair", "long hair"], #index: 7 attribute: hair
                ["no", "yes"], #index: 8 attribute: hat
                ["male","female"], #index: 9 attribute: gender
                ["black", "white", "red", "purple", "yellow", "gray", "blue", "green", "multicolor"], #index: 10 attribute: upcolor
                ["black", "white", "pink", "purple", "yellow", "gray", "blue", "green", "brown", "multicolor"], #index: 11 attribute: downcolor
  ]
  
  for i in range(len(attributes)):
    #le posizioni degli attributi coincidono coi due vettori e per ottenere la label tradotta è sufficente prenderne il valore-1 per ottenere 
    #l'index corrispettivo nella lista di label tradotte
    translated_attributes[attributes_names[i]] = translator[i][attributes[i]-1] 
  return translated_attributes

In [27]:
def get_data(batch_size, img_root):

    # Load data:
    training_data = CustomDataset(
        img_root+"train", True, img_root+"annotations_train.csv")
    val_data = CustomDataset(img_root+"validation",
                             True, img_root+"annotations_train.csv")
    test_data = CustomDataset(img_root+"test", False)
    query_data = CustomDataset(img_root+"queries", False)

    # Initialize dataloaders:
    train_loader = torch.utils.data.DataLoader(
        training_data, batch_size, shuffle=True, num_workers=0)
    val_loader = torch.utils.data.DataLoader(
        val_data, batch_size, shuffle=False, num_workers=0)
    test_loader = torch.utils.data.DataLoader(
        test_data, batch_size, shuffle=False, num_workers=0)
    query_loader = torch.utils.data.DataLoader(
        query_data, batch_size, shuffle=False, num_workers=0)

    return train_loader, val_loader, test_loader, query_loader

**2) CNN implementation**

NOTA: di tutta questa parte va aggiornato completamente il codice con le nostre scelte e reimplementati tutti i classificatori!!! (per task 1 e task 2)

In [28]:
class Our_CNN(torch.nn.Module):
    # init override:
    def __init__(self, num_heads=num_classifiers, loss={'xent'}, **kwargs):

        super(Our_CNN, self).__init__()

        # load default resnet from pytorch:
        resnet = torchvision.models.resnet50(pretrained=True, progress=True)
        # save number of input features from last layer:

        # cavo l'ultimo layer alla resnet
        self.backbone = torch.nn.Sequential(*list(resnet.children())[:-1])

        self.fc = torch.nn.ModuleList()

        self.fc.append(torch.nn.Linear(2048, 4))
        for i in range(1, 10):
            self.fc.append(torch.nn.Linear(2048, 2))
        self.fc.append(torch.nn.Linear(2048, 9))
        self.fc.append(torch.nn.Linear(2048, 10))

    # forward pass:
    def forward(self, x):
        # forward though backbone portion of network:

        x = self.backbone(x)
        x = x.flatten(1)

        # put the output in (batch_size, input_dim) format and save as features:
        feats = x.view(x.shape[0], -1)

        # loop through fc layers and store fwd pass in outputs list:
        outputs = []
        for fc in self.fc:
            outputs.append(fc(x))

        # return both output list (task 1) and features (task 2):
        return outputs, feats


In [29]:
def cost_function(outputs, targets):
    loss = 0.0
    for i in range(len(outputs)):
        loss += F.cross_entropy(outputs[i], targets.t()[i])
    return loss

In [30]:
#Optimizer:
#Da -> https://colab.research.google.com/drive/1oOKXtUUaACKG_P1SZO-PpfygP5_12dX7#scrollTo=FK8A9alWqYZ2 

#Da aggiornare sulla base dei nostri classificatori.
def get_optimizer(model, lr, wd, momentum):
  
  return torch.optim.Adam(
        model.parameters(),
        lr=lr,
        weight_decay=wd)

In [31]:
#TRAIN & TEST FUNCTIONS:
#Da -> https://colab.research.google.com/drive/1oOKXtUUaACKG_P1SZO-PpfygP5_12dX7#scrollTo=FK8A9alWqYZ2 

#Train function:
def train(net,data_loader,optimizer):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  net.train() # Strictly needed if network contains layers which has different behaviours between train and test
  for batch_idx, (inputs, targets) in enumerate(data_loader):
    # Load data into GPU
    inputs = inputs.to(device)
    targets = targets.to(device)
    # Forward pass
    outputs,feats = net(inputs)
    loss = 0

    loss = cost_function(outputs,targets)

    #stats print:
    samples+=inputs.shape[0]

    cumulative_loss += loss.item()

    predicted = []
    for i in range(len(outputs)):
      predicted.append(outputs[i].max(1)[1])
      cumulative_accuracy += predicted[i].eq(targets.t()[i]).sum().item()
      
    # Backward pass
    loss.backward()
    
    # Update parameters
    optimizer.step()
    
    # Reset the optimizer
    optimizer.zero_grad()
  return cumulative_loss/samples, 100*cumulative_accuracy/(len(outputs)*samples)

#Test function:
def test(net, data_loader):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  net.eval() # Strictly needed if network contains layers which has different behaviours between train and test
  with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(data_loader):

      #print("batch_idx: ",batch_idx)
      # Load data into GPU
      inputs = inputs.to(device)
      targets = targets.to(device)

      # Forward pass
      outputs,feats = net(inputs)
      
      # Apply the loss:
      loss = 0
      #for i in range(inputs.shape[0]):
       # layer_loss = cost_function(outputs[i], targets[i])
        # loss = loss + layer_loss
      loss = cost_function(outputs,targets)
      samples+=inputs.shape[0]
      cumulative_loss += loss.item() # Note: the .item() is needed to extract scalars from tensors
      predicted = []
      for i in range(len(outputs)):
        predicted.append(outputs[i].max(1)[1])
        cumulative_accuracy += predicted[i].eq(targets.t()[i]).sum().item()

  return cumulative_loss/samples, 100*cumulative_accuracy/(len(outputs)*samples)

In [32]:
#Funzione di esportazione file output per task 1 ("classification_test.csv" - vedi PDF assignment)

In [33]:
#Funzione di esportazione file output per task 2 ("reid_text.txt" - vedi PDF assignment)

In [34]:
#MAIN PER TASK 1
#Modificare da -> https://colab.research.google.com/drive/1oOKXtUUaACKG_P1SZO-PpfygP5_12dX7#scrollTo=FK8A9alWqYZ2 
#MAIN PER TASK 1
'''
Input arguments
  batch_size: Size of a mini-batch
  device: GPU where you want to train your network
  weight_decay: Weight decay co-efficient for regularization of weights
  momentum: Momentum for SGD optimizer
  epochs: Number of epochs for training the network
  num_classes: Number of classes in your dataset
  visualization_name: Name of the visualization folder
  img_root: The root folder of images
'''

def main(batch_size=64, 
         learning_rate=0.001, 
         weight_decay=0.000001, 
         momentum=0.9, 
         epochs=50, 
         num_classes=12, 
         visualization_name='resnet50', 
         img_root=localpath):
  
  dataset_preparation(img_root,img_root+"annotations_train.csv")#function that prepare the validation, query_validation and test folders
  writer = SummaryWriter(log_dir="runs/exp1")

  # Instantiates dataloaders
  train_loader, val_loader, test_loader, query_loader = get_data(batch_size=batch_size, img_root=img_root)
  
  # Instantiates the model
  print(torch.cuda.get_device_name(0))
  net = Our_CNN()
  net = net.to(device)
  
  # Instantiates the optimizer
  optimizer = get_optimizer(net, learning_rate, weight_decay, momentum)
  
  # Instantiates the cost function
  #cost_function = get_cost_function()

  print('Before training:')
  train_loss, train_accuracy = test(net, train_loader)
  val_loss, val_accuracy = test(net,val_loader)

  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('\t Val loss {:.5f}, Val accuracy {:.2f}'.format(val_loss, val_accuracy))
  print('-----------------------------------------------------')
  
  # Add values to plots
  writer.add_scalar('Loss/train_loss', train_loss, 0)
  writer.add_scalar('Loss/val_loss', val_loss, 0)
  writer.add_scalar('Accuracy/train_accuracy', train_accuracy, 0)
  writer.add_scalar('Accuracy/val_accuracy', val_accuracy, 0)

  for e in range(epochs):
    train_loss, train_accuracy = train(net, train_loader, optimizer)
    val_loss, val_accuracy = test(net, val_loader)
    print('Epoch: {:d}'.format(e+1))
    print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    print('\t Val loss {:.5f}, Val accuracy {:.2f}'.format(val_loss, val_accuracy))
    print('-----------------------------------------------------')
    
    # Add values to plots
    writer.add_scalar('Loss/train_loss', train_loss, e + 1)
    writer.add_scalar('Loss/val_loss', val_loss, e + 1)
    writer.add_scalar('Accuracy/train_accuracy', train_accuracy, e + 1)
    writer.add_scalar('Accuracy/val_accuracy', val_accuracy, e + 1)

  print('After training:')
  train_loss, train_accuracy = test(net, train_loader)
  val_loss, val_accuracy = test(net, val_loader)

  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('\t Val loss {:.5f}, Val accuracy {:.2f}'.format(val_loss, val_accuracy))
  print('-----------------------------------------------------')

  # Closes the logger
  writer.close()

main()


<_io.TextIOWrapper name='/content/Dataset/annotations_train.csv' mode='r' encoding='UTF-8'>
Moving 25% of training files in validation
Training files moved to validation
Tesla K80
Before training:
	 Training loss 0.19159, Training accuracy 46.87
	 Val loss 0.19205, Val accuracy 46.91
-----------------------------------------------------
Epoch: 1
	 Training loss 0.09549, Training accuracy 80.33
	 Val loss 0.11555, Val accuracy 78.04
-----------------------------------------------------
Epoch: 2
	 Training loss 0.07056, Training accuracy 85.52
	 Val loss 0.10659, Val accuracy 80.69
-----------------------------------------------------
Epoch: 3
	 Training loss 0.05844, Training accuracy 88.16
	 Val loss 0.11763, Val accuracy 80.89
-----------------------------------------------------
Epoch: 4
	 Training loss 0.04840, Training accuracy 90.14
	 Val loss 0.11966, Val accuracy 80.86
-----------------------------------------------------
Epoch: 5
	 Training loss 0.03996, Training accuracy 92.01

In [35]:
#MAIN PER TASK 2
#Ripetere da sopra e poi???